# Negative sampling

In [3]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import sklearn.metrics       as Metrics
import pandas                as pd
import matplotlib            as mpl
import random
from calendar import monthrange

# import folium  
# from folium.plugins import HeatMap
# import branca

In [4]:
import json
import requests
import os
import calendar
import datetime

In [5]:
# set matplotlib defaults
%matplotlib inline
sns.set()
plt.rcParams["figure.figsize"] = (15,6)
mpl.rc('axes', labelsize=18)
mpl.rc('xtick', labelsize=16)
mpl.rc('ytick', labelsize=16)
plt.rc('figure', titlesize=18)
plt.style.use('seaborn-darkgrid')

## Data loading and preparation

In [6]:
# read dataset.accidents
data_folder = '../data'
accidents_file = os.path.join(data_folder, 'siniestros_bogota_historico.csv')
accidents = pd.read_csv(accidents_file,sep=',')
accidents.head()

,X,Y,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,ANO_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD_GEO,LOCALIDAD,FECHA_HORA_ACC,CIV,PK_CALZADA
0,-74.039690,4.759877,1001,A00728405,360471,2010,AK 18A-CL 182 58,SOLO DANOS,CHOQUE,USAQUEN,USAQUEN,2010-05-25T20:10:00.000Z,1007130.0,91019224.0
1,-74.114296,4.615476,1002,A00408972,305838,2008,TR 49-3 02,SOLO DANOS,CHOQUE,PUENTE ARANDA,PUENTE ARANDA,2008-09-30T07:00:00.000Z,16004607.0,195361.0
2,-74.105296,4.509792,1003,A000341903,4437952,2016,CL 80A-KR 1 SE 02,CON HERIDOS,CHOQUE,USME,USME,2016-02-27T16:20:00.000Z,5002841.0,149316.0
3,-74.167225,4.631051,1004,A000554581,4472304,2017,AV AVENIDA CIUDAD DE CALI-CL 42 S 02,CON HERIDOS,ATROPELLO,KENNEDY,KENNEDY,2017-02-09T16:45:00.000Z,8004447.0,66217.0
4,-74.121790,4.603106,1005,A000763908,4512837,2018,AV AVENIDA PRIMERA DE MAYO-KR 50A 14,CON HERIDOS,CHOQUE,PUENTE ARANDA,PUENTE ARANDA,2018-03-25T12:10:00.000Z,16003102.0,239094.0


In [7]:
# read dataset.weather
data_folder = '../data'
weather_data = os.path.join(data_folder, 'total.csv')
weather = pd.read_csv(weather_data,sep=',')
weather.head()

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,...,windBearing,cloudCover,uvIndex,visibility,location,pressure,windGust,precipType,ozone,precipAccumulation
0,0,2017-10-27 05:00:00,Clear,clear-night,0.0,0.0,7.0,6.22,6.99,1.00,...,320.0,0.00,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN
1,1,2017-10-27 06:00:00,Clear,clear-night,0.0,0.0,7.0,6.22,6.00,0.93,...,320.0,0.13,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN
2,2,2017-10-27 07:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,7.0,6.22,6.99,1.00,...,0.0,0.44,0.0,10.003,bosa,NaN,NaN,NaN,NaN,NaN
3,3,2017-10-27 08:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,9.0,8.51,9.00,1.00,...,0.0,0.44,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN
4,4,2017-10-27 09:00:00,Mostly Cloudy,partly-cloudy-night,0.0,0.0,10.0,10.00,9.99,1.00,...,251.0,0.75,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN


In [8]:
weather.columns

Index(['Unnamed: 0', 'time', 'summary', 'icon', 'precipIntensity',
       'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint',
       'humidity', 'windSpeed', 'windBearing', 'cloudCover', 'uvIndex',
       'visibility', 'location', 'pressure', 'windGust', 'precipType', 'ozone',
       'precipAccumulation'],
      dtype='object')

In [9]:
weather['timestamp'] = weather.apply(lambda x: datetime.datetime.strptime(x['time'], '%Y-%m-%d %H:%M:%S'), axis = 1)

In [10]:
locations=list(weather.location.unique())
[i.upper() for i in locations]

['BOSA',
 'ENGATIVA',
 'TUNJUELITO',
 'KENNEDY',
 'FONTIBON',
 'CHAPINERO',
 'CIUDADBOLIVAR',
 'ANTONIO NARIÑO',
 'LA CANDELARIA',
 'PUENTE ARANDA',
 'RAFAEL URIBE',
 'BARRIOS UNIDOS',
 'SUBA',
 'SAN CRISTOBAL',
 'SUMAPAZ',
 'LOS MARTIRES',
 'SANTAFE',
 'TEUSAQUILLO',
 'USAQUEN',
 'USME']

In [11]:
# Column renaming
cols = ['x', 'y', 'object_id', 'form', 'accident_id', 'year' , 
        'address', 'severity', 'accident_type', 'borough_geo', 
        'borough', 'timestamp', 'civ', 'pk']
accidents.columns = cols
accidents.timestamp = pd.to_datetime(accidents.timestamp, infer_datetime_format=True)

In [12]:
accidents['date'] = accidents.timestamp.dt.date
accidents['month'] = accidents.timestamp.dt.month
accidents['hour'] = accidents.timestamp.dt.hour

In [13]:
accidents['day'] = accidents.timestamp.dt.day

In [36]:
accidents['x'] = round(accidents['x'], 6)
accidents['y'] = round(accidents['y'], 6)

In [14]:
accidents.head(3)

,x,y,object_id,form,accident_id,year,address,severity,accident_type,borough_geo,borough,timestamp,civ,pk,date,month,hour,day
0,-74.039690,4.759877,1001,A00728405,360471,2010,AK 18A-CL 182 58,SOLO DANOS,CHOQUE,USAQUEN,USAQUEN,2010-05-25 20:10:00+00:00,1007130.0,91019224.0,2010-05-25,5,20,25
1,-74.114296,4.615476,1002,A00408972,305838,2008,TR 49-3 02,SOLO DANOS,CHOQUE,PUENTE ARANDA,PUENTE ARANDA,2008-09-30 07:00:00+00:00,16004607.0,195361.0,2008-09-30,9,7,30
2,-74.105296,4.509792,1003,A000341903,4437952,2016,CL 80A-KR 1 SE 02,CON HERIDOS,CHOQUE,USME,USME,2016-02-27 16:20:00+00:00,5002841.0,149316.0,2016-02-27,2,16,27


In [37]:
accidents_2019=accidents[accidents['year']==2019].reset_index(drop=True)
accidents_2019.head()

,x,y,object_id,form,accident_id,year,address,severity,accident_type,borough_geo,borough,timestamp,civ,pk,date,month,hour,day
0,-74.134363,4.579297,327001,A000908434,10472514,2019,KR 26-CL 52B S 2,CON HERIDOS,CHOQUE,TUNJUELITO,TUNJUELITO,2019-01-11 14:20:00+00:00,6001128.0,232187.0,2019-01-11,1,14,11
1,-74.048819,4.665144,327002,A000908387,10472553,2019,CL 84-KR 8 2,SOLO DANOS,CHOQUE,CHAPINERO,CHAPINERO,2019-01-11 11:30:00+00:00,2000813.0,35700.0,2019-01-11,1,11,11
2,-74.173301,4.615589,327003,A000908334,10472522,2019,KR 80-CL 53A S 2,CON HERIDOS,CHOQUE,KENNEDY,KENNEDY,2019-01-11 23:20:00+00:00,8007967.0,245233.0,2019-01-11,1,23,11
3,-74.025068,4.725288,327004,A000907795,10472569,2019,KR 7-CL 150 2,SOLO DANOS,CHOQUE,USAQUEN,USAQUEN,2019-01-11 06:30:00+00:00,1003533.0,172513.0,2019-01-11,1,6,11
4,-74.080770,4.670083,327005,A000908264,10472515,2019,CL 68-KR 58 2,CON HERIDOS,CHOQUE,BARRIOS UNIDOS,BARRIOS UNIDOS,2019-01-10 07:20:00+00:00,12002817.0,224800.0,2019-01-10,1,7,10


In [39]:
pos=accidents_2019[['x','y','timestamp', 'date', 'month', 'day', 'hour']]
len(pos)
pos.head()

,x,y,timestamp,date,month,day,hour
0,-74.134363,4.579297,2019-01-11 14:20:00+00:00,2019-01-11,1,11,14
1,-74.048819,4.665144,2019-01-11 11:30:00+00:00,2019-01-11,1,11,11
2,-74.173301,4.615589,2019-01-11 23:20:00+00:00,2019-01-11,1,11,23
3,-74.025068,4.725288,2019-01-11 06:30:00+00:00,2019-01-11,1,11,6
4,-74.080770,4.670083,2019-01-10 07:20:00+00:00,2019-01-10,1,10,7


In [38]:
# pos['x'] = round(pos['x'], 6)
# pos['y'] = round(pos['y'], 6)

In [18]:
pos['coords'] = pos[['x', 'y']].apply(lambda x: ', '.join((str(x['x']), str(x['y']))), axis = 1)

# pos['coords'] = pos[['x', 'y']].apply(lambda x: ', '.join((str(round(x['x'], 6)), str(round(x['y'], 6)))), axis = 1)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
pos

,x,y,timestamp,date,month,day,hour,coords
0,-74.134363,4.579297,2019-01-11 14:20:00+00:00,2019-01-11,1,11,14,"-74.134363, 4.579297"
1,-74.048819,4.665144,2019-01-11 11:30:00+00:00,2019-01-11,1,11,11,"-74.048819, 4.665144"
2,-74.173301,4.615589,2019-01-11 23:20:00+00:00,2019-01-11,1,11,23,"-74.173301, 4.615589"
3,-74.025068,4.725288,2019-01-11 06:30:00+00:00,2019-01-11,1,11,6,"-74.025068, 4.725288"
4,-74.080770,4.670083,2019-01-10 07:20:00+00:00,2019-01-10,1,10,7,"-74.08077, 4.670083"
5,-74.151311,4.572193,2019-01-11 09:20:00+00:00,2019-01-11,1,11,9,"-74.151311, 4.572193"
6,-74.142010,4.667770,2019-01-11 01:00:00+00:00,2019-01-11,1,11,1,"-74.14201, 4.66777"
7,-74.135317,4.599616,2019-01-09 23:48:00+00:00,2019-01-09,1,9,23,"-74.135317, 4.599616"
8,-74.135804,4.588874,2019-01-11 17:38:00+00:00,2019-01-11,1,11,17,"-74.135804, 4.588874"
9,-74.149876,4.687513,2019-01-10 16:30:00+00:00,2019-01-10,1,10,16,"-74.149876, 4.687513"


In [20]:
pos = pos[['coords', 'hour', 'day', 'month']]
pos.head(2)

,coords,hour,day,month
0,"-74.134363, 4.579297",14,11,1
1,"-74.048819, 4.665144",11,11,1


In [21]:
def days_in_month(year, month):
    return monthrange(year, month)[1]

In [22]:
t1 = datetime.datetime.now()

neg_df = pd.DataFrame(columns=['coords', 'hour', 'day', 'month'])
cols = pos.columns.tolist()
neg_list = []

for i in range (200):
# for i in range (len(pos)*3):
    
    #we randomly choose which column to randomize
    randcol = random.randint(0, 3)
    
    #sampling a row at random, which column will be modified
    neg = pos.sample()
    
    #sampling another random row, from which we take the value we're replacing in the potential negative sample ('neg')
    sample = pos.sample()
    neg[cols[randcol]] = sample[cols[randcol]].values[0]
    
    #check if 'neg' is part of positive samples:
    
    ncoords = neg['coords'].values[0]
    nhour = neg['hour'].values[0]
    nday = neg['day'].values[0]
    nmonth = neg['month'].values[0]
    
    if pos.loc[(pos['coords'] == ncoords) & (pos['hour'] == nhour) & (pos['day'] == nday) & (pos['month'] == nmonth)].empty: 

        neg_list.append(neg.values.tolist()[0])
      
    else:
        print('found coincidence/day out of month range')
        print(neg.values.tolist()[0])
    
    i += 1
    
neg_df = pd.DataFrame.from_records(neg_list, columns= ['coords', 'hour', 'day', 'month'])
t2 = datetime.datetime.now()

print(t2-t1)

found coincidence/day out of month range
['-74.070175, 4.604504', 13, 14, 3]
found coincidence/day out of month range
['-74.106684, 4.682308', 19, 25, 7]
found coincidence/day out of month range
['-74.117852, 4.680568', 7, 24, 4]
found coincidence/day out of month range
['-74.066862, 4.636441', 7, 14, 9]
found coincidence/day out of month range
['-74.188234, 4.636346', 18, 17, 2]
found coincidence/day out of month range
['-74.042172, 4.775209', 13, 3, 8]
found coincidence/day out of month range
['-74.150146, 4.617477', 14, 30, 7]
found coincidence/day out of month range
['-74.081442, 4.720515', 16, 10, 3]
0:00:02.644449


In [23]:
neg_df

,coords,hour,day,month
0,"-74.106542, 4.549809",10,14,9
1,"-74.097564, 4.637799",11,23,11
2,"-74.084945, 4.589338",12,9,11
3,"-74.156486, 4.618785",14,3,6
4,"-74.120879, 4.696506",4,18,3
5,"-74.087022, 4.722932",21,24,4
6,"-74.101594, 4.596576",13,30,7
7,"-74.13214, 4.709141",6,6,8
8,"-74.048865, 4.738826",19,29,11
9,"-74.109623, 4.663118",12,11,3


In [24]:
neg_df['day_out_of_month_range'] = neg_df.apply(lambda x: x['day'] > days_in_month(2019, x['month']), axis = 1)

In [25]:
#example of a negative sample which 'day' value makes no sense given its 'month' value
neg_df[neg_df['day_out_of_month_range'] == True].head()

,coords,hour,day,month,day_out_of_month_range
91,"-74.077613, 4.657924",20,31,9,True


In [26]:
#removing such samples
neg_df = neg_df[neg_df['day_out_of_month_range'] == False]

In [27]:
neg_df['timestamp'] = neg_df.apply(lambda x: datetime.datetime(2019, x['month'], x['day'], x['hour']), axis = 1)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
neg_df

,coords,hour,day,month,day_out_of_month_range,timestamp
0,"-74.106542, 4.549809",10,14,9,False,2019-09-14 10:00:00
1,"-74.097564, 4.637799",11,23,11,False,2019-11-23 11:00:00
2,"-74.084945, 4.589338",12,9,11,False,2019-11-09 12:00:00
3,"-74.156486, 4.618785",14,3,6,False,2019-06-03 14:00:00
4,"-74.120879, 4.696506",4,18,3,False,2019-03-18 04:00:00
5,"-74.087022, 4.722932",21,24,4,False,2019-04-24 21:00:00
6,"-74.101594, 4.596576",13,30,7,False,2019-07-30 13:00:00
7,"-74.13214, 4.709141",6,6,8,False,2019-08-06 06:00:00
8,"-74.048865, 4.738826",19,29,11,False,2019-11-29 19:00:00
9,"-74.109623, 4.663118",12,11,3,False,2019-03-11 12:00:00


In [29]:
neg_df.drop(columns = ['day_out_of_month_range'], inplace = True)

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
neg_df

,coords,hour,day,month,timestamp
0,"-74.106542, 4.549809",10,14,9,2019-09-14 10:00:00
1,"-74.097564, 4.637799",11,23,11,2019-11-23 11:00:00
2,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00
3,"-74.156486, 4.618785",14,3,6,2019-06-03 14:00:00
4,"-74.120879, 4.696506",4,18,3,2019-03-18 04:00:00
5,"-74.087022, 4.722932",21,24,4,2019-04-24 21:00:00
6,"-74.101594, 4.596576",13,30,7,2019-07-30 13:00:00
7,"-74.13214, 4.709141",6,6,8,2019-08-06 06:00:00
8,"-74.048865, 4.738826",19,29,11,2019-11-29 19:00:00
9,"-74.109623, 4.663118",12,11,3,2019-03-11 12:00:00


In [31]:
#recovering 'x' and 'y' from 'coords'
neg_df['x'] = neg_df.apply(lambda x: float(x['coords'].split(", ")[0]), axis = 1)
neg_df['y'] = neg_df.apply(lambda x: float(x['coords'].split(", ")[1]), axis = 1)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
neg_df

,coords,hour,day,month,timestamp,x,y
0,"-74.106542, 4.549809",10,14,9,2019-09-14 10:00:00,-74.106542,4.549809
1,"-74.097564, 4.637799",11,23,11,2019-11-23 11:00:00,-74.097564,4.637799
2,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338
3,"-74.156486, 4.618785",14,3,6,2019-06-03 14:00:00,-74.156486,4.618785
4,"-74.120879, 4.696506",4,18,3,2019-03-18 04:00:00,-74.120879,4.696506
5,"-74.087022, 4.722932",21,24,4,2019-04-24 21:00:00,-74.087022,4.722932
6,"-74.101594, 4.596576",13,30,7,2019-07-30 13:00:00,-74.101594,4.596576
7,"-74.13214, 4.709141",6,6,8,2019-08-06 06:00:00,-74.132140,4.709141
8,"-74.048865, 4.738826",19,29,11,2019-11-29 19:00:00,-74.048865,4.738826
9,"-74.109623, 4.663118",12,11,3,2019-03-11 12:00:00,-74.109623,4.663118


In [40]:
negatives = neg_df.merge(accidents_2019[['borough', 'x', 'y']], on = ['x', 'y'])

In [41]:
negatives

,coords,hour,day,month,timestamp,x,y,borough
0,"-74.106542, 4.549809",10,14,9,2019-09-14 10:00:00,-74.106542,4.549809,RAFAEL URIBE URIBE
1,"-74.106542, 4.549809",10,14,9,2019-09-14 10:00:00,-74.106542,4.549809,RAFAEL URIBE URIBE
2,"-74.097564, 4.637799",11,23,11,2019-11-23 11:00:00,-74.097564,4.637799,TEUSAQUILLO
3,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,SAN CRISTOBAL
4,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,SANTA FE
5,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,SANTA FE
6,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,CANDELARIA
7,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,SANTA FE
8,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,SANTA FE
9,"-74.084945, 4.589338",12,9,11,2019-11-09 12:00:00,-74.084945,4.589338,SAN CRISTOBAL


In [151]:
weather

,Unnamed: 0,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,...,cloudCover,uvIndex,visibility,location,pressure,windGust,precipType,ozone,precipAccumulation,timestamp
0,0,2017-10-27 05:00:00,Clear,clear-night,0.0000,0.00,7.00,6.22,6.99,1.00,...,0.00,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 05:00:00
1,1,2017-10-27 06:00:00,Clear,clear-night,0.0000,0.00,7.00,6.22,6.00,0.93,...,0.13,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 06:00:00
2,2,2017-10-27 07:00:00,Partly Cloudy,partly-cloudy-night,0.0000,0.00,7.00,6.22,6.99,1.00,...,0.44,0.0,10.003,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 07:00:00
3,3,2017-10-27 08:00:00,Partly Cloudy,partly-cloudy-night,0.0000,0.00,9.00,8.51,9.00,1.00,...,0.44,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 08:00:00
4,4,2017-10-27 09:00:00,Mostly Cloudy,partly-cloudy-night,0.0000,0.00,10.00,10.00,9.99,1.00,...,0.75,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 09:00:00
5,5,2017-10-27 10:00:00,Partly Cloudy,partly-cloudy-night,0.0000,0.00,9.00,9.00,8.00,0.93,...,0.44,0.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 10:00:00
6,6,2017-10-27 11:00:00,Partly Cloudy,partly-cloudy-day,0.0000,0.00,9.00,7.95,8.00,0.93,...,0.44,0.0,10.003,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 11:00:00
7,7,2017-10-27 12:00:00,Partly Cloudy,partly-cloudy-day,0.0000,0.00,12.00,12.00,10.99,0.94,...,0.44,1.0,8.005,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 12:00:00
8,8,2017-10-27 13:00:00,Partly Cloudy,partly-cloudy-day,0.0000,0.00,14.00,14.00,10.99,0.82,...,0.44,3.0,10.003,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 13:00:00
9,9,2017-10-27 14:00:00,Partly Cloudy,partly-cloudy-day,0.0000,0.00,18.00,18.00,9.99,0.59,...,0.44,5.0,10.003,bosa,NaN,NaN,NaN,NaN,NaN,2017-10-27 14:00:00


In [140]:
# neg_df_weather = neg_df.merge(weather, left_on = 'timestamp', right_on = 'timestamp')

In [142]:
# neg_df_weather